<a href="https://colab.research.google.com/github/LCP2022/FL2023/blob/main/RNNCentralisedLocal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install tensorflow
!pip install tensorflow_datasets
!pip install matplotlib

In [5]:
import tensorflow_datasets as tfds
from tensorflow import keras
import tensorflow as tf

import matplotlib.pyplot as plt
import numpy as np

In [6]:
def getCifarData():
    (traindataset,testDataset),datasetinfo = tfds.load(
        'cifar10',
        split=['train','test'],
        shuffle_files= True,
        as_supervised= True,
        with_info= True
    )
    return traindataset,testDataset,datasetinfo

In [7]:
def dataset_to_numpy(dataset):
    features = []
    labels = []
    for image, label in tfds.as_numpy(dataset):
        features.append(image)
        labels.append(label)
    return np.array(features), np.array(labels)

def Splitset(intrain,intest):
    Train_images, Train_label = dataset_to_numpy(intrain)
    Test_images, Test_label = dataset_to_numpy(intest)
    return  Train_images, Train_label,Test_images, Test_label

def datanorm(data):
    return data/255

In [8]:
def split_intoPartition(data, num_partitions):
    # Calculate the size of each partition
    partition_size = len(data) // num_partitions
    # Split the dataset into partitions
    partitions = [data[i * partition_size:(i + 1) * partition_size] for i in range(num_partitions)]
    return partitions

In [9]:
trainset,testset, info = getCifarData()
Train_images, Train_label,Test_images, Test_label = Splitset(trainset,testset)
Train_images_n = datanorm(Train_images)
Test_images_n = datanorm(Test_images)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incomplete8JTEY3/cifar10-train.tfrecord*...:   0%|          …

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incomplete8JTEY3/cifar10-test.tfrecord*...:   0%|          |…

Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


In [10]:
def simpleRNNModel():
    model = keras.Sequential()
    model.add(keras.layers.Reshape((32, 32 * 3), input_shape=(32, 32, 3)))
    model.add(keras.layers.LSTM(128))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(10,activation="softmax"))
    model.compile(optimizer=tf.keras.optimizers.Adam(),loss= "sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

In [11]:
model = simpleRNNModel()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 32, 96)            0         
                                                                 
 lstm (LSTM)                 (None, 128)               115200    
                                                                 
 flatten (Flatten)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 10)                1290      
                                                                 
Total params: 116490 (455.04 KB)
Trainable params: 116490 (455.04 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
history = model.fit(Train_images_n,
                    Train_label,
                    epochs=1000,
                    verbose=1,
                    batch_size=32,
                    )

Epoch 1/1000
1563/1563 [==============================] - 10s 5ms/step - loss: 1.7702 - accuracy: 0.3502
Epoch 2/1000
1563/1563 [==============================] - 8s 5ms/step - loss: 1.5328 - accuracy: 0.4444
Epoch 3/1000
1563/1563 [==============================] - 8s 5ms/step - loss: 1.4243 - accuracy: 0.4833
Epoch 4/1000
1563/1563 [==============================] - 7s 4ms/step - loss: 1.3448 - accuracy: 0.5149
Epoch 5/1000
1563/1563 [==============================] - 8s 5ms/step - loss: 1.2841 - accuracy: 0.5363
Epoch 6/1000
1563/1563 [==============================] - 7s 5ms/step - loss: 1.2346 - accuracy: 0.5557
Epoch 7/1000
1563/1563 [==============================] - 7s 5ms/step - loss: 1.1846 - accuracy: 0.5734
Epoch 8/1000
1563/1563 [==============================] - 8s 5ms/step - loss: 1.1447 - accuracy: 0.5884
Epoch 9/1000
1563/1563 [==============================] - 7s 5ms/step - loss: 1.1105 - accuracy: 0.6006
Epoch 10/1000
1563/1563 [==============================] - 8s 5

In [13]:
loss,acc = model.evaluate(Test_images_n,Test_label)

313/313 [==============================] - 1s 3ms/step - loss: 2.7523 - accuracy: 0.5500


In [14]:
print(round(acc*100,2))

55.0
